In [ ]:
import torch as th

BW = 50 # Mbits/s
FPS = 15
PSZ = 4 * 16 # 4 channels of 16 bits each

MFS = BW/FPS # Maximum Frame Size in Mbits


# a tensor 480x480x4  pxl, pxl,(x,y,z,validity)
a = th.rand(480, 480, 4)*20

a[:,:,3] = (th.rand(480, 480) > 0).float()

a = a.view(-1, 4).contiguous()

n_points = a.size(0)


bins = th.tensor([0, 3, 6, 9, 10])

n_areas = bins.size(0)

tic = th.cuda.Event(enable_timing=True)
toc = th.cuda.Event(enable_timing=True)



areas = th.bucketize(a[:, 2].contiguous(), bins)

# print(areas.shape)
# print(n_areas)

area_mask = th.zeros((n_points, n_areas), dtype=th.bool, device=a.device)
area_mask[th.arange(n_points), areas-1] = 1

# print(area_mask)


# print(area_mask)
# apply validity mask to columns of area_mask
validity_mask = a[:, 3].bool()
area_mask = area_mask & validity_mask.unsqueeze(1)


#print(area_mask)
# cumsum per column

area_counts = area_mask.cumsum(dim=0)[-1, :]
#print(area_counts)

valid_points = area_counts.sum().item()
print(valid_points, "are the ", valid_points/n_points*100, "% of the points")




frame_size = valid_points * PSZ

print("frame size", frame_size/1000000, "Mbits")

print("maximum frame size", MFS)
print(frame_size*FPS/1000000, "Mbits/s required for,", FPS, "FPS")
print("BW available:", BW, "Mbits/s")
print("with this BW we can send", MFS*1000000/PSZ, "points per frame")

print("needed compression factor", (MFS*1000000/PSZ)/valid_points)




230400 are the  100.0 % of the points
7.136064052581787 ms
frame size 14.7456 Mbits
maximum frame size 3.3333333333333335
221.184 Mbits/s required for, 15 FPS
BW available: 50 Mbits/s
with this BW we can send 52083.333333333336 points per frame
needed compression factor 0.22605613425925927


In [2]:
# # or all columns

# retain_mask = area_mask.any(dim=1).squeeze()

# print(retain_mask)  

# E = X[retain_mask]

# print(E)